In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# --- Example usage ---
# Initialize with base prompt from Cell 0
PREDS_DICT = {
    "eczema": 0.78,
    "contact_dermatitis": 0.15,
    "psoriasis": 0.04,
    "tinea_corporis": 0.02,
    "seborrheic_dermatitis": 0.01
}

METADATA = {
    "user_input": "I’ve had a red, itchy patch on the inside of my left elbow for about two weeks. It gets worse after I use scented soap or take a hot shower. The skin feels dry and slightly scaly but not painful or bleeding. I haven’t had anything like this before and haven’t used any creams yet.",
    "cv_analysis": {
        "area": 8.4,
        "color_profile": {
            "average_Lab": [67.2, 18.4, 9.3],
            "redness_index": 0.34,
            "texture_contrast": 0.12
        }
    },
    "history": {
        "2025-10-10": {
            "cv_analysis": {
                "area": 12.6,
                "color_profile": {
                    "average_Lab": [67.2, 18.4, 9.3],
                    "redness_index": 0.31,
                    "texture_contrast": 0.13
                }
            },
            "text_summary": "Bro is still struggling with dermatitis"
        },
        "2025-10-20": {
            "text_summary": "Bro is struggling with dermatitis. He's been using a cream but it's not helping."
        },
        "2025-11-04": {
            "cv_analysis": {
                "area": 8.4,
                "color_profile": {
                    "average_Lab": [69.2, 16.4, 9.3],
                    "redness_index": 0.21,
                    "texture_contrast": 0.13
                }
            }
        }

    }
}

QUESTION_1 = "What cream should I use?"
QUESTION_2 = "But fragnance free creams are expensive"

In [23]:
# Test APIManager class
import sys
import os
from pathlib import Path

# Add the frontend-react/python directory to path to import APIManager
# Notebook is in src/llm/, api_manager is in src/frontend-react/python/
# Go up one level from llm to src, then into frontend-react/python
current_dir = Path(os.getcwd())

# Find the src directory (could be current_dir or current_dir.parent)
if (current_dir / "frontend-react" / "python" / "api_manager.py").exists():
    # We're in src/
    api_manager_path = current_dir / "frontend-react" / "python"
elif (current_dir.parent / "frontend-react" / "python" / "api_manager.py").exists():
    # We're in src/llm/
    api_manager_path = current_dir.parent / "frontend-react" / "python"
else:
    # Try from project root
    api_manager_path = current_dir / "src" / "frontend-react" / "python"

sys.path.insert(0, str(api_manager_path))
print(f"✓ Added to path: {api_manager_path}")

from api_manager import APIManager

# Initialize APIManager in dummy mode for testing
case_id = "test_case_simulate"
manager = APIManager(case_id, dummy=True)

# Test initial prediction
print("\n--- Testing get_initial_prediction ---")
dummy_image = "/Users/tk20/Desktop/APCOMP215/ac215_Group_127_2473879/src/llm/test_case_simulate/img1.png"  # Placeholder image
from PIL import Image
image = Image.open(dummy_image)
text_description = METADATA["user_input"]

results = manager.get_initial_prediction(
    image=image,
    text_description=text_description,
    user_timestamp=None  # Will auto-generate if None
)

print("\n✓ Initial Prediction Results:")
print(f"  LLM Response (first 150 chars): {results['llm_response'][:150]}...")
print(f"  Top Prediction: {max(results['predictions'].items(), key=lambda x: x[1])}")
print(f"  CV Analysis Area: {results['cv_analysis']['area']}")
print(f"  CV Redness Index: {results['cv_analysis']['color_profile']['redness_index']}")

# Test follow-up chat
print("\n--- Testing chat_message (Follow-up 1) ---")
chat_response1 = manager.chat_message(
    user_query=QUESTION_1,
    user_timestamp=None  # Will auto-generate if None
)

print(f"\n✓ Chat Response 1:")
print(f"  Question: {QUESTION_1}")
print(f"  Answer (first 150 chars): {chat_response1['answer'][:150]}...")
print(f"  Conversation History Length: {len(chat_response1['conversation_history'])}")

# Test second follow-up
print("\n--- Testing chat_message (Follow-up 2) ---")
chat_response2 = manager.chat_message(
    user_query=QUESTION_2,
    user_timestamp=None  # Will auto-generate if None
)

print(f"\n✓ Chat Response 2:")
print(f"  Question: {QUESTION_2}")
print(f"  Answer (first 150 chars): {chat_response2['answer'][:150]}...")
print(f"  Conversation History Length: {len(chat_response2['conversation_history'])}")
print(f"  Full History: {chat_response2['conversation_history']}")



✓ Added to path: /Users/tk20/Desktop/APCOMP215/ac215_Group_127_2473879/src/frontend-react/python
/Users/tk20/Desktop/APCOMP215/ac215_Group_127_2473879/src/llm/test_case_simulate

--- Testing get_initial_prediction ---
Processing initial prediction for case test_case_simulate...
  → Saving image...
  → Running local ML model for embeddings...
    [DUMMY MODE] Returning dummy embeddings...
  → Running CV analysis...
    [DUMMY MODE] Returning dummy CV analysis...
  → Getting cloud predictions...
    [DUMMY MODE] Returning dummy predictions...
  → Calling LLM for explanation...
 Calling LLM explain API...
  → Saving conversation...
  → Saving history...
✓ Initial prediction complete for case test_case_simulate

✓ Initial Prediction Results:
  LLM Response (first 150 chars): It sounds like you might be experiencing eczema, which is a common skin condition causing itchy, red, and dry patches. Based on what you've described,...
  Top Prediction: ('eczema', 0.78)
  CV Analysis Area: 8.4
  CV 

In [20]:
results

{'llm_response': "It sounds like you might be dealing with eczema, which is a common skin condition causing itchy, red patches. Based on what you've described, the patch on your elbow seems to be quite red and covers a noticeable area.\n\nThe image and your history suggest a flare-up of irritated skin. It's good you noticed it gets worse with scented soap and hot showers, as those are common triggers. The dryness and scaling fit with eczema, and it's helpful that it's not painful or bleeding right now.\n\nEczema often happens because of a combination of things. Sometimes it's related to genetics, meaning it runs in families. Other times, it's triggered by things that irritate the skin, like certain soaps, detergents, or even weather changes. Stress can also sometimes play a role.\n\nTo help soothe the skin at home, try using a gentle, fragrance-free cleanser instead of your usual soap. After bathing, gently pat your skin dry and immediately apply a thick, plain moisturizer to help lock

In [21]:
import requests

url = "https://tanushkmr2001--dermatology-llm-27b-dermatologyllm-explain-dev.modal.run"
payload = {"predictions": PREDS_DICT,
           "metadata": METADATA}

initial_response = requests.post(url, json=payload)

print(initial_response.status_code)
print(initial_response.json())  # or response.text if not JSON


200
It sounds like you might be dealing with eczema, which is a common skin condition that causes red, itchy patches. Based on what you've described and the images, it looks like the patch on your elbow has gotten a bit smaller and less red since it first appeared, but it's still causing you discomfort.

Eczema often flares up when the skin gets dry or comes into contact with things that irritate it. Things like scented soaps, hot water, certain fabrics, or even stress can sometimes trigger it. It's also quite common for it to appear in the creases of elbows and knees.

To help soothe the area at home, try using a gentle, fragrance-free cleanser instead of scented soap, and keep showers lukewarm rather than hot. Gently pat your skin dry afterwards and immediately apply a thick, fragrance-free moisturizer to help lock in moisture. You might also find wearing soft cotton clothing helps prevent irritation.

While it seems to be improving slightly, it's a good idea to see a doctor if the p

In [22]:
payload_question = {
    "initial_answer": response.json(),
    "question": "What cream should I use?"
}
url = 'https://tanushkmr2001--dermatology-llm-27b-dermatologyllm-as-a085de-dev.modal.run'

response_question1 = requests.post(url, json=payload_question)
print(response_question1.status_code)
print(response_question1.json()['answer'])

200
Okay, let's talk about creams! Since it looks like eczema, and you haven't used anything yet, starting with a gentle, fragrance-free moisturizer is a great first step. Look for something with ceramides or hyaluronic acid – these ingredients help repair the skin barrier.

Applying it regularly, especially after bathing, can really help keep the skin hydrated and reduce the itchiness. If that doesn't provide enough relief after a week or two, you might consider an over-the-counter hydrocortisone cream (like 1%) for a short period to calm the inflammation. Just use it sparingly as directed on the package.

Remember, if things don't improve or get worse, it's always best to check in with a doctor or dermatologist. They can give you a specific diagnosis and recommend the most effective treatment for your skin.


In [24]:
payload_question = {
    "initial_answer": response.json(),
    "question": "But fragnance free creams are expensive",
    "conversation_history": response_question1.json()['conversation_history']
}
url = 'https://tanushkmr2001--dermatology-llm-27b-dermatologyllm-as-a085de-dev.modal.run'

response_question2 = requests.post(url, json=payload_question)
print(response_question2.status_code)
print(response_question2.json()['answer'])

200
Okay, I understand. Finding fragrance-free creams can sometimes feel a bit pricey, and it's frustrating when you're trying to take care of your skin!

Since we think this might be eczema, and you've noticed scented soaps make it worse, avoiding fragrances in your moisturizers is a really good step. While some fragrance-free options can be more expensive, look for simpler ingredients. Sometimes, basic, thick ointments (like plain petroleum jelly) or creams designed for sensitive skin are effective and more budget-friendly.

You could also try applying your moisturizer right after a lukewarm bath or shower while your skin is still slightly damp to help lock in moisture. Remember, consistency is key! If cost is a major barrier, definitely mention that to a doctor – they might have specific, affordable recommendations or be able to prescribe something if needed.


In [2]:
# Test APIManager class
import sys
import os
from pathlib import Path

# Add the frontend-react/python directory to path to import APIManager
# Notebook is in src/llm/, api_manager is in src/frontend-react/python/
# Go up one level from llm to src, then into frontend-react/python
current_dir = Path(os.getcwd())

# Find the src directory (could be current_dir or current_dir.parent)
if (current_dir / "frontend-react" / "python" / "api_manager.py").exists():
    # We're in src/
    api_manager_path = current_dir / "frontend-react" / "python"
elif (current_dir.parent / "frontend-react" / "python" / "api_manager.py").exists():
    # We're in src/llm/
    api_manager_path = current_dir.parent / "frontend-react" / "python"
else:
    # Try from project root
    api_manager_path = current_dir / "src" / "frontend-react" / "python"

sys.path.insert(0, str(api_manager_path))
print(f"✓ Added to path: {api_manager_path}")

from api_manager import APIManager

# Initialize APIManager in dummy mode for testing
case_id = "test_case_simulate"
manager = APIManager(case_id, dummy=True)

# Test initial prediction
print("\n--- Testing get_initial_prediction ---")
dummy_image = "path/to/test_image.jpg"  # Placeholder image
text_description = METADATA["user_input"]

results = manager.get_initial_prediction(
    image=dummy_image,
    text_description=text_description,
    user_timestamp=None  # Will auto-generate if None
)

print("\n✓ Initial Prediction Results:")
print(f"  LLM Response (first 150 chars): {results['llm_response'][:150]}...")
print(f"  Top Prediction: {max(results['predictions'].items(), key=lambda x: x[1])}")
print(f"  CV Analysis Area: {results['cv_analysis']['area']}")
print(f"  CV Redness Index: {results['cv_analysis']['color_profile']['redness_index']}")

# Test follow-up chat
print("\n--- Testing chat_message (Follow-up 1) ---")
chat_response1 = manager.chat_message(
    user_query=QUESTION_1,
    user_timestamp=None  # Will auto-generate if None
)

print(f"\n✓ Chat Response 1:")
print(f"  Question: {QUESTION_1}")
print(f"  Answer (first 150 chars): {chat_response1['answer'][:150]}...")
print(f"  Conversation History Length: {len(chat_response1['conversation_history'])}")

# Test second follow-up
print("\n--- Testing chat_message (Follow-up 2) ---")
chat_response2 = manager.chat_message(
    user_query=QUESTION_2,
    user_timestamp=None  # Will auto-generate if None
)

print(f"\n✓ Chat Response 2:")
print(f"  Question: {QUESTION_2}")
print(f"  Answer (first 150 chars): {chat_response2['answer'][:150]}...")
print(f"  Conversation History Length: {len(chat_response2['conversation_history'])}")
print(f"  Full History: {chat_response2['conversation_history']}")




✓ Added to path: /Users/tk20/Desktop/APCOMP215/ac215_Group_127_2473879/src/frontend-react/python
/Users/tk20/Desktop/APCOMP215/ac215_Group_127_2473879/src/llm/test_case_simulate

--- Testing get_initial_prediction ---
Processing initial prediction for case test_case_simulate...
  → Saving image...
  → Running local ML model for embeddings...
    [DUMMY MODE] Returning dummy embeddings...
  → Running CV analysis...
    [DUMMY MODE] Returning dummy CV analysis...
  → Getting cloud predictions...
    [DUMMY MODE] Returning dummy predictions...
  → Calling LLM for explanation...
    [DUMMY MODE] Returning dummy LLM explanation...
  → Saving conversation...
    [DUMMY MODE] Skipping conversation save
  → Saving history...
    [DUMMY MODE] Skipping history save
✓ Initial prediction complete for case test_case_simulate

✓ Initial Prediction Results:
  LLM Response (first 150 chars): It sounds like you might be dealing with eczema, which is a common skin condition that causes red, itchy patche